In [ ]:
!pip install tensorflow tensorflow-model-optimization torch transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import time

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt')

start_time = time.time()
outputs = model(**inputs)
end_time = time.time()
print(f"Baseline Inference Time: {end_time - start_time} seconds")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Baseline Inference Time: 0.8300719261169434 seconds


In [ ]:
from torch.nn.utils import prune
import torch.nn as nn

def apply_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

pruned_model = apply_pruning(model, amount=0.4)

start_time = time.time()
outputs = pruned_model(**inputs)
end_time = time.time()
print(f"Pruned Model Inference Time: {end_time - start_time} seconds")


Pruned Model Inference Time: 0.08072662353515625 seconds


In [ ]:
from transformers import AdamW

optimizer = AdamW(pruned_model.parameters(), lr=1e-5)

pruned_model.train()
for epoch in range(3):
    for _ in range(100):
        outputs = pruned_model(**inputs)
        loss = outputs.last_hidden_state.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

pruned_model.eval()
start_time = time.time()
outputs = pruned_model(**inputs)
end_time = time.time()
print(f"Fine-Tuned Pruned Model Inference Time: {end_time - start_time} seconds")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss: -0.014200284145772457
Epoch 0, Loss: -0.014070943929255009
Epoch 0, Loss: -0.015664463862776756
Epoch 0, Loss: -0.016274521127343178
Epoch 0, Loss: -0.01824636198580265
Epoch 0, Loss: -0.021059831604361534
Epoch 0, Loss: -0.021320147439837456
Epoch 0, Loss: -0.02207319438457489
Epoch 0, Loss: -0.02450956590473652
Epoch 0, Loss: -0.02544860541820526
Epoch 0, Loss: -0.025416282936930656
Epoch 0, Loss: -0.02681928314268589
Epoch 0, Loss: -0.02824246697127819
Epoch 0, Loss: -0.02793026529252529
Epoch 0, Loss: -0.027826743200421333
Epoch 0, Loss: -0.02740151435136795
Epoch 0, Loss: -0.029871240258216858
Epoch 0, Loss: -0.02983921580016613
Epoch 0, Loss: -0.030281024053692818
Epoch 0, Loss: -0.030539030209183693
Epoch 0, Loss: -0.030804714187979698
Epoch 0, Loss: -0.03024018369615078
Epoch 0, Loss: -0.031153442338109016
Epoch 0, Loss: -0.031578872352838516
Epoch 0, Loss: -0.031513866037130356
Epoch 0, Loss: -0.03243841230869293
Epoch 0, Loss: -0.03276079520583153
Epoch 0, Loss

KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, inputs):
    start_time = time.time()
    outputs = model(**inputs)
    end_time = time.time()
    latency = end_time - start_time
    return latency, outputs

baseline_latency, _ = evaluate_model(model, inputs)
print(f"Baseline Latency: {baseline_latency}")

pruned_latency, _ = evaluate_model(pruned_model, inputs)
print(f"Pruned Latency: {pruned_latency}")


Baseline Latency: 0.157426118850708
Pruned Latency: 0.1251077651977539


In [ ]:
!pip install torch transformers
import torch
from transformers import BertTokenizer, BertModel
import time
import torch.quantization as tq

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = pruned_model

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt')



In [ ]:
model.qconfig = tq.get_default_qat_qconfig('fbgemm')

tq.prepare_qat(model, inplace=True)

print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(
      30522, 768, padding_idx=0
      (activation_post_process): FusedMovingAvgObsFakeQuantize(
        fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
        (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
      )
    )
    (position_embeddings): Embedding(
      512, 768
      (activation_post_process): FusedMovingAvgObsFakeQuantize(
        fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
        (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
      )
    )
    (token_type_embeddings): Embedding

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

model.train()
for epoch in range(3):
    for _ in range(100):
        outputs = model(**inputs)
        loss = outputs.last_hidden_state.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

tq.convert(model, inplace=True)

model.eval()
start_time = time.time()
outputs = model(**inputs)
end_time = time.time()
print(f"Quantized Model Inference Time: {end_time - start_time} seconds")


Epoch 0, Loss: -0.043984491378068924
Epoch 0, Loss: -0.0437280535697937
Epoch 0, Loss: -0.04430490732192993
Epoch 0, Loss: -0.04511970654129982
Epoch 0, Loss: -0.0452011339366436
Epoch 0, Loss: -0.045800644904375076
Epoch 0, Loss: -0.045629408210515976
Epoch 0, Loss: -0.045682549476623535
Epoch 0, Loss: -0.046334464102983475
Epoch 0, Loss: -0.04587974771857262
Epoch 0, Loss: -0.045548874884843826
Epoch 0, Loss: -0.045791249722242355


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, inputs):
    start_time = time.time()
    outputs = model(**inputs)
    end_time = time.time()
    latency = end_time - start_time
    return latency, outputs

baseline_latency, _ = evaluate_model(model, inputs)
print(f"Baseline Latency: {baseline_latency}")

pruned_latency, _ = evaluate_model(pruned_model, inputs)
print(f"Pruned Latency: {pruned_latency}")

quantized_latency, _ = evaluate_model(model, inputs)
print(f"Quantized Latency: {quantized_latency}")

#code fucked up, refined below this


Baseline Latency: 2.869445562362671
Pruned Latency: 4.29597806930542
Quantized Latency: 3.1121857166290283


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.utils import prune
import torch.nn as nn
import time
import torch.quantization as tq

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt')

start_time = time.time()
outputs = model(**inputs)
end_time = time.time()
print(f"Baseline Inference Time: {end_time - start_time} seconds")

def refined_pruning(model, amount=0.2):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

pruned_model = refined_pruning(model, amount=0.2)

start_time = time.time()
outputs = pruned_model(**inputs)
end_time = time.time()
print(f"Pruned Model Inference Time: {end_time - start_time} seconds")

quantized_model = tq.quantize_dynamic(
    pruned_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

start_time = time.time()
outputs = quantized_model(**inputs)
end_time = time.time()
print(f"Quantized Model Inference Time: {end_time - start_time} seconds")


Baseline Inference Time: 0.1383683681488037 seconds
Pruned Model Inference Time: 0.08554625511169434 seconds
Quantized Model Inference Time: 0.06628870964050293 seconds


In [ ]:
!pip install datasets
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.utils import prune
import torch.nn as nn
import time
import torch.quantization as tq
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt')

start_time = time.time()
outputs = model(**inputs)
end_time = time.time()
print(f"Baseline Inference Time: {end_time - start_time} seconds")

def refined_pruning(model, amount=0.2):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

pruned_model = refined_pruning(model, amount=0.2)

start_time = time.time()
outputs = pruned_model(**inputs)
end_time = time.time()
print(f"Pruned Model Inference Time: {end_time - start_time} seconds")

quantized_model = tq.quantize_dynamic(
    pruned_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

start_time = time.time()
outputs = quantized_model(**inputs)
end_time = time.time()
print(f"Quantized Model Inference Time: {end_time - start_time} seconds")

classification_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

def preprocess_function(examples):
    return tokenizer(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        padding='max_length',
        max_length=44
    )

encoded_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

trainer = Trainer(
    model=classification_model,
    args=training_args,
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=lambda p: metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids),
)

print("Evaluating the original model...")
original_results = trainer.evaluate()

pruned_classification_model = refined_pruning(classification_model, amount=0.2)
quantized_classification_model = tq.quantize_dynamic(
    pruned_classification_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

trainer.model = quantized_classification_model

print("Evaluating the pruned and quantized model...")
pruned_quantized_results = trainer.evaluate()

print("Original model results:", original_results)
print("Pruned and quantized model results:", pruned_quantized_results)


Baseline Inference Time: 0.12452316284179688 seconds
Pruned Model Inference Time: 0.13669776916503906 seconds
Quantized Model Inference Time: 0.048059701919555664 seconds


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Evaluating the original model...


Evaluating the pruned and quantized model...
Original model results: {'eval_loss': 0.8054389953613281, 'eval_accuracy': 0.3137254901960784, 'eval_f1': 0.0, 'eval_runtime': 73.6556, 'eval_samples_per_second': 5.539, 'eval_steps_per_second': 0.692}
Pruned and quantized model results: {'eval_loss': 0.8124847412109375, 'eval_accuracy': 0.3161764705882353, 'eval_f1': 0.0, 'eval_runtime': 43.3961, 'eval_samples_per_second': 9.402, 'eval_steps_per_second': 1.175}


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.quantization as tq
from transformers import BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from torch.nn.utils import prune
import time

#create the custom lightweight "student" model

class CustomLightweightModel(nn.Module):
    def __init__(self):
        super(CustomLightweightModel, self).__init__()
        self.embedding = nn.Embedding(30522, 128)
        self.conv1 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 128, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, input_ids, labels=None):
        x = self.embedding(input_ids).transpose(1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        logits = self.fc2(x)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.fc2.out_features), labels.view(-1))
            return loss, logits
        else:
            return logits

#set that as the new model
model = CustomLightweightModel()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#https://huggingface.co/docs/datasets/v1.13.2/load_hub.html use huggingface
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)
#create a custom trainer & checkk loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        outputs = model(inputs['input_ids'], labels=labels)
        loss = outputs[0]
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=lambda p: metric.compute(predictions=torch.argmax(torch.tensor(p.predictions), axis=1), references=p.label_ids),
)

trainer.train()
#prune the model might drop acc tho
def apply_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv1d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

pruned_model = apply_pruning(model, amount=0.4)

#quantized the model
quantized_model = tq.quantize_dynamic(
    pruned_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=128)

quantized_model.eval()
start_time = time.time()
with torch.no_grad():
    outputs = quantized_model(inputs['input_ids'])
end_time = time.time()
print(f"Quantized Model Inference Time: {end_time - start_time} seconds")

print("Evaluating the quantized model...")
trainer.model = quantized_model
quantized_results = trainer.evaluate()

print("Quantized model results:", quantized_results)





/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.647900,0.614900,0.683824,0.812227


Quantized Model Inference Time: 0.020845651626586914 seconds
Evaluating the quantized model...


Quantized model results: {'eval_loss': 0.6169583797454834, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.45, 'eval_samples_per_second': 906.695, 'eval_steps_per_second': 113.337, 'epoch': 1.0}
